# 3에서 바뀐 부분

생명선 라벨링 수정 후 다시

In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tqdm.auto import tqdm

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
label_mapping = {
    '000': 0,
    '001': 1,
    '010': 2,
    '011': 3,
    '100': 4,
    '101': 5,
    '110': 6,
    '111': 7
}

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
%cd /content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data

/content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data


In [19]:
df = pd.read_excel('label3.xlsx')
df = df[~df['label'].str.contains("-")]  # Filter out invalid entries
df['label'] = df['label'].map(label_mapping)  # Map string labels to integers

In [20]:
print(df)

      File Number  life head  heart  label
0               1     0    0      0      0
1               2     0    0      0      0
2               5     0    1      0      2
3              12     0    0      1      1
4              13     0    1      0      2
...           ...   ...  ...    ...    ...
1030         5353     0    0      1      1
1031         5354     1    0      1      5
1032         5366     0    0      0      0
1033         5367     0    0      0      0
1035         5385     0    0      1      1

[996 rows x 5 columns]


In [21]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [22]:
def load_and_preprocess_images(image_df):
    images = []
    for idx, row in tqdm(image_df.iterrows(), total=image_df.shape[0]):
        image_path = os.path.join('./Mask', f"image{row['File Number']}.png")
        image = Image.open(image_path).resize((128, 128))
        image = np.array(image) / 255.0
        if image.ndim == 2 or image.shape[2] == 1:
            image = np.stack((image,)*3, axis=-1)  # Convert grayscale to RGB
        images.append(image)
    return np.array(images)

In [23]:
train_images = load_and_preprocess_images(train_df)
test_images = load_and_preprocess_images(test_df)

  0%|          | 0/796 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [24]:
train_labels = to_categorical(train_df['label'], num_classes=8)
test_labels = to_categorical(test_df['label'], num_classes=8)

In [25]:
# Initialize MobileNetV2 model with pretrained ImageNet weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base model

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='softmax')  # 8 classes for the final layer
])

9406464/9406464 [==============================] - 0s 0us/step


In [26]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
# Callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    ModelCheckpoint('best_model.h5', save_best_only=True, save_weights_only=True)
]

history = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=50,  # May increase if model is not overfitting
    callbacks=callbacks
)

# Load the best weights and unfreeze the base_model for fine-tuning
model.load_weights('best_model.h5')
base_model.trainable = True

Epoch 1/50
25/25 [==============================] - 16s 534ms/step - loss: 2.0393 - accuracy: 0.2412 - val_loss: 1.6934 - val_accuracy: 0.3750
Epoch 2/50
25/25 [==============================] - 10s 388ms/step - loss: 1.7708 - accuracy: 0.3530 - val_loss: 1.6009 - val_accuracy: 0.4000
Epoch 3/50
25/25 [==============================] - 11s 428ms/step - loss: 1.6809 - accuracy: 0.3668 - val_loss: 1.4377 - val_accuracy: 0.4650
Epoch 4/50
25/25 [==============================] - 12s 493ms/step - loss: 1.5546 - accuracy: 0.4284 - val_loss: 1.4021 - val_accuracy: 0.4800
Epoch 5/50
25/25 [==============================] - 10s 390ms/step - loss: 1.4806 - accuracy: 0.4384 - val_loss: 1.3720 - val_accuracy: 0.5000
Epoch 6/50
25/25 [==============================] - 12s 470ms/step - loss: 1.4560 - accuracy: 0.4736 - val_loss: 1.3511 - val_accuracy: 0.4800
Epoch 7/50
25/25 [==============================] - 14s 582ms/step - loss: 1.3720 - accuracy: 0.4912 - val_loss: 1.2942 - val_accuracy: 0.5200

In [28]:
# Recompile the model with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

7/7 [==============================] - 4s 230ms/step - loss: 1.0501 - accuracy: 0.6050
Test accuracy: 60.50%


In [30]:
# Fine-tune the model
history_fine = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
25/25 [==============================] - 59s 2s/step - loss: 5.8284 - accuracy: 0.1545 - val_loss: 1.0613 - val_accuracy: 0.6150
Epoch 2/20
25/25 [==============================] - 42s 2s/step - loss: 5.0959 - accuracy: 0.1935 - val_loss: 1.1793 - val_accuracy: 0.5300
Epoch 3/20
25/25 [==============================] - 42s 2s/step - loss: 4.5281 - accuracy: 0.1784 - val_loss: 1.4526 - val_accuracy: 0.4950
Epoch 4/20
25/25 [==============================] - 43s 2s/step - loss: 3.8937 - accuracy: 0.2337 - val_loss: 1.9295 - val_accuracy: 0.3500
Epoch 5/20
25/25 [==============================] - 43s 2s/step - loss: 3.3595 - accuracy: 0.2601 - val_loss: 2.5652 - val_accuracy: 0.2650
Epoch 6/20
25/25 [==============================] - 41s 2s/step - loss: 2.9938 - accuracy: 0.2688 - val_loss: 3.2231 - val_accuracy: 0.1900
Epoch 6: early stopping


In [31]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

7/7 [==============================] - 1s 204ms/step - loss: 3.2231 - accuracy: 0.1900
Test accuracy: 19.00%
